In [1]:
import pymongo
import json
import requests
import os
from dotenv import load_dotenv


In [2]:
load_dotenv()
API_KEY = os.getenv('KEY')
DB_CONNECTION = os.getenv('DB_string')

In [3]:
# Establish a connection with mongoDB
client = pymongo.MongoClient(DB_CONNECTION)
dataBase = client['BetMe']

In [4]:
COLLECTION_NAME = "Uk_Odds"
Odds_uk = dataBase[COLLECTION_NAME]


For updating the odds of in-play matches i have gathered the data and updated it everytime by removing previous collection
because if there might be mismatch of records in the API and database. Hence to remove tha confusion i'm removing the previous
collection and updating it every time. 

In [5]:
def Uk_odds():
    
    if "Uk_Odds" not in dataBase.list_collection_names():
        Odds_uk = dataBase.create_collection("Uk_Odds")
    else:
        sport_key = 'upcoming'
        odds_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
            'api_key': API_KEY,
            'sport': sport_key,
            'region': 'uk', 
            'mkt': 'h2h', 
            'dateFormat': 'iso'})
    
    return odds_response




In [6]:
response = Uk_odds()            


In [7]:
in_play_data = json.loads(response.text)

Odds_uk.delete_many({})
Odds_uk.insert_many(in_play_data['data'])

The tasks below are to display the in-play matches in UK region. I have made a final data that gives information of League name,
sports, current odds of different bookmaker

In [8]:
import pandas as pd


In [9]:
data = pd.DataFrame(in_play_data['data'])

In [10]:
data.head()

,_id,commence_time,home_team,id,sites,sites_count,sport_key,sport_nice,teams
0,60522a6cf63d65f7924f0701,2021-03-17T15:05:04Z,FC Rotor Volgograd,9c8b35ba72ebc6b637f618fd18a879e5,"[{'site_key': 'paddypower', 'site_nice': 'Padd...",4,soccer_russia_premier_league,Premier League - Russia,"[FC Rotor Volgograd, FK Rostov]"
1,60522a6cf63d65f7924f0702,2021-03-17T17:00:00Z,FC Akhmat Grozny,8c5f47e067b78c0b97e7b322b728e550,"[{'site_key': 'ladbrokes', 'site_nice': 'Ladbr...",9,soccer_russia_premier_league,Premier League - Russia,"[Arsenal Tula, FC Akhmat Grozny]"
2,60522a6cf63d65f7924f0703,2021-03-17T17:00:00Z,CSKA Moscow,1f1c1ab3b564423ea04533eb273145e8,"[{'site_key': 'ladbrokes', 'site_nice': 'Ladbr...",9,soccer_russia_premier_league,Premier League - Russia,"[CSKA Moscow, Zenit St Petersburg]"
3,60522a6cf63d65f7924f0704,2021-03-17T17:05:00Z,Boston Red Sox,249ad457d7af97b7b6b00c465a850b95,"[{'site_key': 'paddypower', 'site_nice': 'Padd...",3,baseball_mlb,MLB,"[Boston Red Sox, Minnesota Twins]"
4,60522a6cf63d65f7924f0705,2021-03-17T17:05:00Z,Philadelphia Phillies,c1ec6fd81d77a7df539f87cae706668d,"[{'site_key': 'paddypower', 'site_nice': 'Padd...",3,baseball_mlb,MLB,"[Detroit Tigers, Philadelphia Phillies]"


In [11]:
odds = []

for i in range(len(data['sites'])):
    for j in range(len(data['sites'][i])):
        odds.append(data['sites'][i][j]['odds'])
        
    

In [12]:
sites = []
for i in range(len(data['sites'])):
    for j in range(len(data['sites'][i])):
        sites.append(data['sites'][i][j]['site_key'])
        
    

In [13]:
m = []
teams = []
for i in range(len(data)):
    l = [data.teams[i]] * data.sites_count[i]
    m.append(l)
    
for j in range(len(m)):
    for i in m[j]:
        teams.append(i)                   
            

In [14]:
m = []
leagues = []
for i in range(len(data)):
    l = [data.sport_key[i]] * data.sites_count[i]
    m.append(l)
    
for j in range(len(m)):
    for i in m[j]:
        leagues.append(i)                   
            

In [15]:
Final_data = pd.DataFrame({'Leagues': leagues, 'Team': teams, 'Sites':sites, 'Odds': odds})

In [17]:
Final_data

,Leagues,Team,Sites,Odds
0,soccer_russia_premier_league,"[FC Rotor Volgograd, FK Rostov]",paddypower,"{'h2h': [14.0, 1.22, 4.5]}"
1,soccer_russia_premier_league,"[FC Rotor Volgograd, FK Rostov]",betway,"{'h2h': [13.0, 1.25, 4.5]}"
2,soccer_russia_premier_league,"[FC Rotor Volgograd, FK Rostov]",betfair,"{'h2h': [19.0, 1.34, 4.8], 'h2h_lay': [20.0, 1..."
3,soccer_russia_premier_league,"[FC Rotor Volgograd, FK Rostov]",unibet,"{'h2h': [21.0, 1.24, 4.5]}"
4,soccer_russia_premier_league,"[Arsenal Tula, FC Akhmat Grozny]",ladbrokes,"{'h2h': [4.0, 1.87, 3.0]}"
5,soccer_russia_premier_league,"[Arsenal Tula, FC Akhmat Grozny]",paddypower,"{'h2h': [4.2, 1.85, 3.1]}"
6,soccer_russia_premier_league,"[Arsenal Tula, FC Akhmat Grozny]",marathonbet,"{'h2h': [4.33, 1.96, 3.52]}"
7,soccer_russia_premier_league,"[Arsenal Tula, FC Akhmat Grozny]",betway,"{'h2h': [4.33, 1.88, 3.25]}"
8,soccer_russia_premier_league,"[Arsenal Tula, FC Akhmat Grozny]",betfair,"{'h2h': [4.8, 1.96, 3.45], 'h2h_lay': [4.9, 1...."
9,soccer_russia_premier_league,"[Arsenal Tula, FC Akhmat Grozny]",unibet,"{'h2h': [4.1, 1.92, 3.2]}"
